# K₇ Spectral Gap — Proper Method (v3)

**Key fixes from G2_Universality analysis:**
1. ✅ Normalized Laplacian: L = I - D⁻¹/² W D⁻¹/²
2. ✅ Geodesic S³ distance with factor 2: d = 2·arccos(|q₁·q₂|)
3. ✅ Topological metric: det(g) = 65/32 hardcoded
4. ✅ TCS ratio from topology: r = H*/84
5. ✅ Calibration on S³ (λ₁ = 3 exact)

**No PINN needed** — the metric is topologically determined.

---

**Target**: λ₁ × H* = 13 (or 14 = dim(G₂))

In [ ]:
# §1 Setup
import numpy as np
import json
import time
import os
from datetime import datetime

os.makedirs('outputs', exist_ok=True)

# GPU
try:
    import cupy as cp
    import cupyx.scipy.sparse as cp_sparse
    import cupyx.scipy.sparse.linalg as cp_linalg
    GPU = True
    xp = cp
    print(f"✓ GPU: {cp.cuda.runtime.getDeviceProperties(0)['name'].decode()}")
except:
    GPU = False
    xp = np
    print("⚠ CPU mode")

# Constants
DIM_G2 = 14
B2, B3 = 21, 77
H_STAR = B2 + B3 + 1  # = 99
DET_G = 65 / 32       # = 2.03125

print(f"H* = {H_STAR}, det(g) = {DET_G}")

In [ ]:
# §2 Quaternion Utilities

def sample_S3(n, seed=None):
    """Sample n points uniformly on S³."""
    if seed is not None:
        np.random.seed(seed)
    x = np.random.randn(n, 4)
    return x / np.linalg.norm(x, axis=1, keepdims=True)

def geodesic_S3(Q1, Q2=None):
    """
    Geodesic distance on S³.
    
    d(q₁, q₂) = 2 × arccos(|q₁·q₂|)
    
    The factor 2 is CRITICAL: S³ has diameter π, not π/2.
    """
    if Q2 is None:
        Q2 = Q1
    dot = np.abs(Q1 @ Q2.T)
    dot = np.clip(dot, 0, 1)
    return 2.0 * np.arccos(dot)  # Factor 2!

print("✓ Quaternion utilities")

In [ ]:
# §3 Normalized Graph Laplacian

def normalized_laplacian(D, k=25):
    """
    Build normalized graph Laplacian from distance matrix.
    
    L = I - D^(-1/2) W D^(-1/2)
    
    Eigenvalues ∈ [0, 2], λ₀ = 0 (constant mode).
    """
    n = D.shape[0]
    
    # Adaptive bandwidth (median of k-NN distances)
    knn_dists = np.sort(D, axis=1)[:, 1:k+1]
    sigma = np.median(knn_dists)
    
    # Gaussian kernel
    W = np.exp(-D**2 / (2 * sigma**2))
    np.fill_diagonal(W, 0)  # No self-loops
    
    # k-NN sparsification
    for i in range(n):
        idx = np.argsort(W[i])[:-k]  # Keep only k largest
        W[i, idx] = 0
    W = (W + W.T) / 2  # Symmetrize
    
    # Degree matrix
    d = np.sum(W, axis=1)
    d_inv_sqrt = np.where(d > 1e-10, 1.0 / np.sqrt(d), 0)
    
    # Normalized Laplacian: L = I - D^(-1/2) W D^(-1/2)
    D_inv_sqrt = np.diag(d_inv_sqrt)
    L = np.eye(n) - D_inv_sqrt @ W @ D_inv_sqrt
    
    return L, sigma

def compute_eigenvalues(L, k=10):
    """Compute k smallest eigenvalues."""
    if GPU:
        L_gpu = cp.asarray(L)
        L_sparse = cp_sparse.csr_matrix(L_gpu)
        eigs, _ = cp_linalg.eigsh(L_sparse, k=k, which='SA')
        return cp.sort(eigs).get()
    else:
        from scipy.sparse.linalg import eigsh
        from scipy.sparse import csr_matrix
        eigs, _ = eigsh(csr_matrix(L), k=k, which='SA')
        return np.sort(eigs)

print("✓ Normalized Laplacian")

In [ ]:
# §4 CALIBRATION: Test on S³ (λ₁ = 3 exact)

print("="*50)
print("CALIBRATION: S³ Spectrum")
print("="*50)
print("S³ exact eigenvalues: λₖ = k(k+2)")
print("So λ₀=0, λ₁=3, λ₂=8, λ₃=15...")
print()

N_cal = 5000
Q_cal = sample_S3(N_cal, seed=42)
D_cal = geodesic_S3(Q_cal)

print(f"N = {N_cal} points on S³")
print(f"Distance matrix: min={D_cal[D_cal>0].min():.4f}, max={D_cal.max():.4f}")

L_cal, sigma_cal = normalized_laplacian(D_cal, k=30)
print(f"Bandwidth σ = {sigma_cal:.4f}")

eigs_cal = compute_eigenvalues(L_cal, k=10)

print("\nNormalized Laplacian eigenvalues:")
for i, e in enumerate(eigs_cal):
    print(f"  λ_{i} = {e:.6f}")

# For normalized Laplacian, eigenvalues are in [0,2]
# We need to rescale: the gap ratio should match
# λ₁/λ₀ in unnormalized = 3/0 → λ₁ is the gap
gap_cal = eigs_cal[1]
print(f"\nSpectral gap (normalized): {gap_cal:.6f}")
print(f"Expected pattern: λ₁ should be well-separated from λ₀≈0")

In [ ]:
# §5 TCS Metric on S¹ × S³ × S³

def tcs_distance_matrix(theta, Q1, Q2, ratio):
    """
    TCS metric distance on S¹ × S³ × S³.
    
    ds² = α dθ² + ds₁² + r² ds₂²
    
    where det(g) = α × 1 × r² = 65/32 (topological constraint)
    
    So α = det_g / r²
    """
    n = len(theta)
    
    # Metric coefficients from topology
    r = ratio
    alpha = DET_G / (r ** 2)
    
    # S¹ distance (periodic)
    dtheta = theta[:, None] - theta[None, :]
    dtheta = np.abs(dtheta)
    dtheta = np.minimum(dtheta, 2*np.pi - dtheta)  # Periodic
    
    # S³ geodesic distances (with factor 2!)
    d1 = geodesic_S3(Q1)
    d2 = geodesic_S3(Q2)
    
    # Combined metric distance
    # D² = α × dθ² + d1² + r² × d2²
    D_sq = alpha * dtheta**2 + d1**2 + (r**2) * d2**2
    D = np.sqrt(D_sq)
    
    return D, {'alpha': alpha, 'r': r, 'det_g': alpha * r**2}

print("✓ TCS metric defined")

In [ ]:
# §6 Main Computation: K₇ Spectral Gap

print("="*60)
print("K₇ SPECTRAL GAP COMPUTATION")
print("="*60)

# Parameters
N = 10000  # Start moderate, can increase
k_neighbors = 40

# TCS ratio formulas to test
ratios = {
    'H*/84 (6×14)': H_STAR / 84,      # = 99/84 ≈ 1.179
    'H*/78 (6×13)': H_STAR / 78,      # = 99/78 ≈ 1.269
    'H*/91 (7×13)': H_STAR / 91,      # = 99/91 ≈ 1.088
    '33/28': 33/28,                    # ≈ 1.179 (same as H*/84)
}

results = {}

for name, ratio in ratios.items():
    print(f"\n--- Testing ratio: {name} = {ratio:.6f} ---")
    
    # Sample TCS neck
    np.random.seed(123)
    theta = 2 * np.pi * np.random.rand(N)
    Q1 = sample_S3(N)
    Q2 = sample_S3(N)
    
    # Compute TCS distance matrix
    t0 = time.time()
    D, params = tcs_distance_matrix(theta, Q1, Q2, ratio)
    print(f"  Distance matrix: {time.time()-t0:.1f}s")
    print(f"  Metric params: α={params['alpha']:.4f}, r={params['r']:.4f}, det(g)={params['det_g']:.6f}")
    
    # Build normalized Laplacian
    t0 = time.time()
    L, sigma = normalized_laplacian(D, k=k_neighbors)
    print(f"  Laplacian: {time.time()-t0:.1f}s, σ={sigma:.4f}")
    
    # Compute eigenvalues
    t0 = time.time()
    eigs = compute_eigenvalues(L, k=15)
    print(f"  Eigenvalues: {time.time()-t0:.1f}s")
    
    # Spectral gap
    lambda_1 = eigs[1]  # Skip λ₀ ≈ 0
    
    # For normalized Laplacian, we need to interpret differently
    # The ratio λ₁_normalized relates to the actual gap
    # Let's compute λ₁ × H* using the normalized value
    
    # Actually, for graph Laplacian on manifold:
    # λ_graph ≈ c × λ_manifold × h² where h = spacing
    # For normalized version, it's more complex
    # Let's just report the raw values and see patterns
    
    print(f"\n  Spectrum: λ₀={eigs[0]:.6f}, λ₁={eigs[1]:.6f}, λ₂={eigs[2]:.6f}")
    
    results[name] = {
        'ratio': ratio,
        'eigenvalues': eigs.tolist(),
        'lambda_1': float(lambda_1),
        'params': params
    }

In [ ]:
# §7 Scale Analysis

print("="*60)
print("SCALE ANALYSIS")
print("="*60)

# The normalized Laplacian eigenvalues are in [0, 2]
# We need to find the scaling to physical eigenvalues

# From calibration on S³:
# - True λ₁(S³) = 3 (exact, for unit S³)
# - Normalized λ₁ = gap_cal

# So scaling factor ≈ 3 / gap_cal for S³
# But this depends on the manifold volume/diameter

print(f"\nS³ calibration: normalized λ₁ = {gap_cal:.6f}")
print(f"S³ true λ₁ = 3")
print(f"Implied scale factor: {3/gap_cal:.4f}")

print("\n" + "-"*40)
print("K₇ Results Summary")
print("-"*40)

for name, res in results.items():
    lam1 = res['lambda_1']
    # If we scale similarly...
    # But TCS has different geometry, so let's just look at ratios
    print(f"\n{name}:")
    print(f"  Normalized λ₁ = {lam1:.6f}")
    print(f"  λ₁/λ₁(S³) = {lam1/gap_cal:.4f}")
    
    # The spectral gap product should be:
    # λ₁(physical) × H* = 13 or 14
    # For normalized Laplacian, we look at relative scaling
    
    # Key insight: the RATIO of λ₁ values for different TCS ratios
    # should reveal the optimal point

In [ ]:
# §8 Scan TCS Ratio

print("="*60)
print("TCS RATIO SCAN")
print("="*60)

# Scan around the expected optimal ratio
ratio_values = np.linspace(0.8, 1.5, 15)
lambda1_values = []

N_scan = 5000  # Faster for scan
k_scan = 30

print(f"Scanning {len(ratio_values)} ratio values with N={N_scan}...\n")

for i, ratio in enumerate(ratio_values):
    np.random.seed(42)
    theta = 2 * np.pi * np.random.rand(N_scan)
    Q1 = sample_S3(N_scan)
    Q2 = sample_S3(N_scan)
    
    D, _ = tcs_distance_matrix(theta, Q1, Q2, ratio)
    L, _ = normalized_laplacian(D, k=k_scan)
    eigs = compute_eigenvalues(L, k=5)
    
    lambda1_values.append(eigs[1])
    print(f"  r={ratio:.3f}: λ₁={eigs[1]:.6f}")

lambda1_values = np.array(lambda1_values)

# Find minimum (spectral gap is minimized at optimal geometry)
# Or maximum depending on convention
idx_min = np.argmin(lambda1_values)
idx_max = np.argmax(lambda1_values)

print(f"\nMin λ₁ at r = {ratio_values[idx_min]:.4f}: λ₁ = {lambda1_values[idx_min]:.6f}")
print(f"Max λ₁ at r = {ratio_values[idx_max]:.4f}: λ₁ = {lambda1_values[idx_max]:.6f}")
print(f"\nExpected optimal: H*/84 = {H_STAR/84:.4f}")

In [ ]:
# §9 High-Resolution at Optimal Ratio

print("="*60)
print("HIGH-RESOLUTION COMPUTATION")
print("="*60)

# Use optimal ratio from theory
optimal_ratio = H_STAR / 84  # = 33/28

N_high = 20000
k_high = 50
n_seeds = 5

print(f"Ratio = H*/84 = {optimal_ratio:.6f}")
print(f"N = {N_high}, k = {k_high}, seeds = {n_seeds}")

lambda1_runs = []

for seed in range(n_seeds):
    print(f"\nSeed {seed+1}/{n_seeds}...")
    t0 = time.time()
    
    np.random.seed(seed * 100)
    theta = 2 * np.pi * np.random.rand(N_high)
    Q1 = sample_S3(N_high)
    Q2 = sample_S3(N_high)
    
    D, params = tcs_distance_matrix(theta, Q1, Q2, optimal_ratio)
    L, sigma = normalized_laplacian(D, k=k_high)
    eigs = compute_eigenvalues(L, k=10)
    
    lambda1_runs.append(eigs[1])
    print(f"  λ₁ = {eigs[1]:.6f} ({time.time()-t0:.1f}s)")

lambda1_mean = np.mean(lambda1_runs)
lambda1_std = np.std(lambda1_runs)

print(f"\n" + "="*40)
print(f"RESULT: λ₁ = {lambda1_mean:.6f} ± {lambda1_std:.6f}")
print(f"="*40)

In [ ]:
# §10 Physical Interpretation

print("="*60)
print("PHYSICAL INTERPRETATION")
print("="*60)

# For normalized Laplacian, eigenvalues are in [0, 2]
# The spectral gap λ₁ relates to mixing time and geometry

# Key insight from G2_Universality:
# They found that λ₁ × H* scales with dim(G₂) - 1 = 13
# But this was for a SPECIFIC construction of the Laplacian

# Let's compute the dimensionless product using our normalization
# λ₁(normalized) × H* 

product = lambda1_mean * H_STAR

print(f"\nλ₁ (normalized) = {lambda1_mean:.6f}")
print(f"H* = {H_STAR}")
print(f"")
print(f"λ₁ × H* = {product:.4f}")
print(f"")
print(f"Target = 13 (dim(G₂) - 1)")
print(f"Deviation = {abs(product - 13)/13 * 100:.1f}%")
print(f"")
print(f"Target = 14 (dim(G₂))")
print(f"Deviation = {abs(product - 14)/14 * 100:.1f}%")

In [ ]:
# §11 Visualization

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# 1. Ratio scan
ax = axes[0]
ax.plot(ratio_values, lambda1_values, 'b.-', lw=2, ms=8)
ax.axvline(H_STAR/84, color='r', ls='--', label=f'H*/84 = {H_STAR/84:.3f}')
ax.axvline(H_STAR/78, color='g', ls='--', label=f'H*/78 = {H_STAR/78:.3f}')
ax.set_xlabel('TCS ratio r')
ax.set_ylabel('λ₁ (normalized)')
ax.set_title('Spectral Gap vs TCS Ratio')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. High-res runs
ax = axes[1]
ax.bar(range(len(lambda1_runs)), lambda1_runs, color='steelblue')
ax.axhline(lambda1_mean, color='r', ls='-', lw=2, label=f'Mean: {lambda1_mean:.4f}')
ax.set_xlabel('Seed')
ax.set_ylabel('λ₁')
ax.set_title(f'High-Resolution Runs (N={N_high})')
ax.legend()

# 3. Result comparison
ax = axes[2]
values = [product, 13, 14]
labels = [f'Measured\n{product:.2f}', 'Target\n13', 'dim(G₂)\n14']
colors = ['steelblue', 'green', 'orange']
bars = ax.bar(labels, values, color=colors)
ax.set_ylabel('λ₁ × H*')
ax.set_title('Spectral Gap Product')
ax.set_ylim(0, 16)

plt.tight_layout()
plt.savefig('outputs/k7_spectral_v3_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Figure saved")

In [ ]:
# §12 Export Results

final_results = {
    'metadata': {
        'timestamp': datetime.now().isoformat(),
        'notebook': 'K7_Spectral_v3_Proper.ipynb',
        'method': 'Normalized Laplacian with topological TCS metric',
    },
    'constants': {
        'H_star': H_STAR,
        'det_g': DET_G,
        'dim_G2': DIM_G2,
    },
    'calibration_S3': {
        'N': N_cal,
        'lambda_1_normalized': float(gap_cal),
        'expected_true_lambda_1': 3,
    },
    'ratio_scan': {
        'ratios': ratio_values.tolist(),
        'lambda1_values': lambda1_values.tolist(),
    },
    'high_resolution': {
        'N': N_high,
        'k': k_high,
        'optimal_ratio': float(optimal_ratio),
        'n_seeds': n_seeds,
        'lambda1_runs': [float(x) for x in lambda1_runs],
        'lambda1_mean': float(lambda1_mean),
        'lambda1_std': float(lambda1_std),
    },
    'final_result': {
        'lambda1_times_Hstar': float(product),
        'target_13': 13,
        'target_14': 14,
        'deviation_from_13_percent': float(abs(product - 13)/13 * 100),
        'deviation_from_14_percent': float(abs(product - 14)/14 * 100),
    }
}

with open('outputs/k7_spectral_v3_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print("✓ Results saved: outputs/k7_spectral_v3_results.json")

In [ ]:
# §13 Final Summary

print("="*70)
print("K₇ SPECTRAL GAP v3 — FINAL SUMMARY")
print("="*70)
print()
print("METHOD")
print("-"*40)
print("  • Normalized Laplacian: L = I - D⁻¹/² W D⁻¹/²")
print("  • S³ geodesic: d = 2·arccos(|q₁·q₂|)")
print("  • TCS metric: det(g) = 65/32 (topological)")
print(f"  • Optimal ratio: r = H*/84 = {H_STAR/84:.6f}")
print()
print("CALIBRATION (S³)")
print("-"*40)
print(f"  Normalized λ₁(S³) = {gap_cal:.6f}")
print(f"  True λ₁(S³) = 3 (exact)")
print()
print("K₇ RESULT")
print("-"*40)
print(f"  λ₁ = {lambda1_mean:.6f} ± {lambda1_std:.6f}")
print(f"  λ₁ × H* = {product:.4f}")
print()
print("COMPARISON")
print("-"*40)
print(f"  Target (dim(G₂)-1) = 13: {abs(product-13)/13*100:.1f}% deviation")
print(f"  Target (dim(G₂))   = 14: {abs(product-14)/14*100:.1f}% deviation")
print()
print("="*70)